In [1]:
# after the wheel added to the resource job config , we don't need to source in the code any more

# import os
# import sys
# current_dir = os.getcwd()
# project_root = os.path.abspath(os.path.join(current_dir, "..", "..", ".."))

In [2]:
# # project_root = os.path.abspath(os.path.join(current_dir, ".."))
# sys.path.append(project_root)
# sys.path

In [ ]:
from citibike.citibike_utils import get_trip_duration_mins
from utils.datetime_utils import timestamp_to_date_col
from pyspark.sql.functions import create_map, lit


In [ ]:
pipeline_id = dbutils.widgets.get("pipeline_id")
run_id = dbutils.widgets.get("run_id")
task_id = dbutils.widgets.get("task_id")
processed_timestamp = dbutils.widgets.get("processed_timestamp")
# catalog = dbutils.widgets.get("catalog")

In [4]:
catalog = 'citibike_dev'

In [5]:
df = spark.read.table(f"{catalog}.01_bronze.jc_citibike")

In [6]:
df = get_trip_duration_mins(spark, df, "started_at", "ended_at", "trip_duration_mins")

In [7]:
df = timestamp_to_date_col(spark, df, "started_at", "trip_start_date")

In [ ]:
# df.first().asDict()

{'ride_id': '29DAF43DD84B4B7A',
 'rideable_type': 'electric_bike',
 'started_at': datetime.datetime(2025, 3, 20, 14, 58, 31, 217000),
 'ended_at': datetime.datetime(2025, 3, 20, 15, 0, 46, 466000),
 'start_station_name': '6 St & Grand St',
 'start_station_id': 'HB302',
 'end_station_name': 'Mama Johnson Field - 4 St & Jackson St',
 'end_station_id': 'HB404',
 'start_lat': Decimal('41'),
 'start_lng': Decimal('-74'),
 'end_lat': Decimal('41'),
 'end_lng': Decimal('-74'),
 'member_casual': 'member',
 'metadata': {'pipeline_id': 'placeholder',
  'run_id': 'placeholder',
  'task_id': 'placeholder',
  'processed_timestamp': 'placeholder'},
 'trip_duration_mins': 2.25,
 'trip_start_date': datetime.date(2025, 3, 20)}

In [ ]:
# df = df.withColumn("metadata", 
#               create_map(
#                   lit("pipeline_id"), lit("placeholder"),
#                   lit("run_id"), lit("placeholder"),
#                   lit("task_id"), lit("placeholder"),
#                   lit("processed_timestamp"), lit("placeholder")
#                   ))

df = df.withColumn("metadata", 
              create_map(
                  lit("pipeline_id"), lit(pipeline_id),
                  lit("run_id"), lit(run_id),
                  lit("task_id"), lit(task_id),
                  lit("processed_timestamp"), lit(processed_timestamp)
                  ))

In [10]:
df = df.select(
    "ride_id",
    "trip_start_date",
    "started_at",
    "ended_at",
    "start_station_name",
    "end_station_name",
    "trip_duration_mins",
    "metadata"
    )

In [ ]:
# df.first().asDict()

{'ride_id': '29DAF43DD84B4B7A',
 'trip_start_date': datetime.date(2025, 3, 20),
 'started_at': datetime.datetime(2025, 3, 20, 14, 58, 31, 217000),
 'ended_at': datetime.datetime(2025, 3, 20, 15, 0, 46, 466000),
 'start_station_name': '6 St & Grand St',
 'end_station_name': 'Mama Johnson Field - 4 St & Jackson St',
 'trip_duration_mins': 2.25,
 'metadata': {'pipeline_id': 'placeholder',
  'run_id': 'placeholder',
  'task_id': 'placeholder',
  'processed_timestamp': 'placeholder'}}

In [12]:
df.write.\
    mode("overwrite").\
    option("overwriteSchema", "true").\
    saveAsTable(f"{catalog}.02_silver.jc_citibike")